<a href="https://colab.research.google.com/github/blufzzz/Introspective-Neural-Networks/blob/master/Introspective_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torchvision
from torch import nn
import torchvision
from torchvision.models import resnet18


In [4]:
model = resnet18(pretrained = True)
X = torch.randn(1,3,256,256)
model.forward(X).shape

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.torch/models/resnet18-5c106cde.pth
46827520it [00:00, 76423530.09it/s]


torch.Size([1, 1000])

In [0]:
class ICNN(nn.Module):
  
  def __init__(self, cnn_model, n_classes = False):
      super(ICNN, self).__init__()
      
      self.conv_features = [] # store conv_features for all passes
      self.model = cnn_model
      self.n_classes = n_classes
      self.sigmoid = nn.Sigmoid()

      self.model.fc = nn.Linear(in_features=512, out_features=1, bias=True)
      
  def forward(self, input, sigmoid = False):
      
      hook = lambda module, input, output: self.conv_features.append(output)
      self.model.avgpool.register_forward_hook(hook)
      output = self.model.forward(input)
      
#       # OVR classification
#       if n_classes:
#           # we took conv_features from the last forward pass
#           output = [fc(self.conv_features[-1]) for fc in self.fc_layers]
      
      if sigmoid:
          return  self.sigmoid(output)
      
      return output


In [7]:
X = torch.randn(2,3,256,256)
icnn_model = ICNN(model)
print(icnn_model.forward(X, sigmoid=True).shape)

torch.Size([2, 1])


In [0]:
batch_size = 16
n=100
X = torch.randn(100, 3, 256, 256)
y = torch.LongTensor(n).random_(2)*2-1
X_1 = X[y==1]
X_2 = X[y==-1]

In [0]:
batch_1 = X_1[torch.LongTensor(batch_size).random_(len(X_1))]
batch_2 = X_2[torch.LongTensor(batch_size).random_(len(X_2))]
alphas = torch.FloatTensor(batch_size).uniform_(0,1)
batch_hat = torch.zeros_like(batch_1,requires_grad=True)
for i in range(batch_size):
  batch_hat[i]=batch_1[i]*alphas[i]+batch_2[i]*(1-alphas[i])

In [0]:
icnn_model=ICNN(model)
f_1 = icnn_model.forward(batch_1, sigmoid=False)
f_2 = icnn_model.forward(batch_2, sigmoid=False)
f_hat = icnn_model.forward(batch_hat, sigmoid=False)

In [0]:
grads_hat = torch.zeros_like(batch_hat)
for i in range(batch_size):
  grads_hat[i]=torch.autograd.grad(f_hat[i],batch_hat,retain_graph=True)[0][i]

In [0]:
loss = 0
for i in range(batch_size):
  loss+=f_2[i]-f_1[i]+torch.norm(grads_hat[i],p=2)

loss/=batch_size

In [0]:
loss.backward()